In [3]:
import pandas as pd
import json
import os

import tqdm

In [1]:
import Config
import Utils
DATA_FILE_PATH = Config.DATA_FILE_PATH_CONTINUOUS_RAW
OUTPUT_FILE_PATH = Config.DATA_FILE_PATH_CONTINUOUS_EXTRACTED
BEGIN_TIME = str(Utils.timestr2timestamp(Config.BEGIN_TIME))
END_TIME = str(Utils.timestr2timestamp(Config.END_TIME))
EXTRACT_INFO_MINUTE = Config.EXTRACT_INFO['minute']
BEGIN_TIME = Utils.timestr2time(Config.BEGIN_TIME)
END_TIME = Utils.timestr2time(Config.END_TIME)

LABEL_FILE_PATH_LIST = [Config.LABEL_FILE_PATH_BEFORE, Config.LABEL_FILE_PATH_AFTER]
LABEL_FILE_PATH_EMA = Config.LABEL_FILE_PATH_EMA
LABEL_FILE_PATH = Config.LABEL_FILE_PATH

### 抽取血氧、心率、运动的有效数据

In [6]:
for file_name, info in EXTRACT_INFO_MINUTE.items():
    print(file_name)
    df = pd.read_csv(
        f"{DATA_FILE_PATH}/{file_name}.csv", encoding="utf-8", dtype=str)[info['useful_cols']]
    # 对时间进行筛选
    df = df[(BEGIN_TIME <= df['recordtime']) & (df['recordtime'] < END_TIME)]
    # 创建输出目录
    try:
        os.mkdir(f"{OUTPUT_FILE_PATH}/{file_name}")
    except:
        ...
    ex_id_list = df['externalid'].unique()
    # 遍历所有external_id
    for ex_id in ex_id_list:
        # 创建当前external_id的列表，用于储存有用数据
        useful_data = []
        # 筛选当前external_id的子表
        df_ex = df[df['externalid'] == ex_id]
        # 遍历子表中每一行的有用数据
        for index, line in df_ex.iterrows():
            extraceded_line = {}
            extractor = EXTRACT_INFO_MINUTE[file_name]['processer']
            useful = json.loads(line[extractor['col_name']].replace("'", '"'))
            for i in range(len(extractor['df_col_name'])):
                if len(extractor['df_col_path'][i]) == 1:
                    extraceded_line[extractor['df_col_name'][i]] = useful.get(extractor['df_col_path'][i][0])
                elif len(extractor['df_col_path'][i]) == 2:
                    extraceded_line[extractor['df_col_name'][i]] = useful.get(extractor['df_col_path'][i][0]).get(extractor['df_col_path'][i][1])
            extraceded_line['timestamp'] = line['recordtime']
            useful_data.append(extraceded_line)
        # 将初步筛选出有效信息的数据以external_id存储到文件
        df_useful = pd.DataFrame(useful_data)
        print(f'ex_id: {ex_id}, count: {len(useful_data)}')
        df_useful.to_csv(f"{OUTPUT_FILE_PATH}/{file_name}/{ex_id}.csv", encoding="utf-8", index=False)
    print()

continuousbloodoxygensaturation
ex_id: 17, count: 7072
ex_id: 58, count: 2751
ex_id: 12, count: 2751
ex_id: 6, count: 11835
ex_id: 50, count: 279
ex_id: 3, count: 2607
ex_id: 47, count: 596
ex_id: 32, count: 8044
ex_id: 29, count: 818
ex_id: 30, count: 639
ex_id: 25, count: 1986
ex_id: 54, count: 7806
ex_id: 5, count: 850
ex_id: 38, count: 942
ex_id: 53, count: 213
ex_id: 55, count: 2495
ex_id: 57, count: 8995
ex_id: 37, count: 7515
ex_id: 34, count: 10428
ex_id: 41, count: 7768
ex_id: 31, count: 1676
ex_id: 44, count: 4878
ex_id: 48, count: 6482
ex_id: 45, count: 5902
ex_id: 15, count: 1011
ex_id: 42, count: 1721
ex_id: 1, count: 1567
ex_id: 56, count: 936
ex_id: 4, count: 5294
ex_id: 14, count: 1324
ex_id: 59, count: 2041
ex_id: 23, count: 11143
ex_id: 11, count: 2894
ex_id: 52, count: 7040
ex_id: 40, count: 1631
ex_id: 8, count: 1
ex_id: 21, count: 1582
ex_id: 49, count: 5274
ex_id: 10, count: 2376
ex_id: 22, count: 201
ex_id: 36, count: 2165
ex_id: 28, count: 7118
ex_id: 13, count:

### 抽取RRI的有效数据

In [6]:
# 采集颗粒度为秒的只有rri
file_name = 'continuousrri'
EXTRACT_INFO_RRI = Config.EXTRACT_INFO['second'][file_name]

df = pd.read_csv(f"{DATA_FILE_PATH}/{file_name}_part11.csv", encoding="utf-8", dtype=str)[EXTRACT_INFO_RRI['useful_cols']]
df.head()

,recordtime,externalid,rriData
0,1663240486000,1224,"[{'rri': {'unit': 'ms', 'value': 1043}, 'sqi':..."
1,1663240546000,1224,"[{'rri': {'unit': 'ms', 'value': 1268}, 'sqi':..."
2,1663240666000,1224,"[{'rri': {'unit': 'ms', 'value': 1346}, 'sqi':..."
3,1663240847000,1224,"[{'rri': {'unit': 'ms', 'value': 1887}, 'sqi':..."
4,1663240910000,1224,"[{'rri': {'unit': 'ms', 'value': 617}, 'sqi': ..."


In [8]:
# 采集颗粒度为秒的只有rri
file_name = 'continuousrri'
EXTRACT_INFO_RRI = Config.EXTRACT_INFO['second'][file_name]

df = pd.read_csv(f"{DATA_FILE_PATH}/{file_name}.csv", encoding="utf-8", dtype=str)[EXTRACT_INFO_RRI['useful_cols']]
# 对时间进行筛选
df = df[(BEGIN_TIME <= df['recordtime']) & (df['recordtime'] < END_TIME)]
# 创建输出目录
try:
    os.mkdir(f"{OUTPUT_FILE_PATH}/{file_name}")
except:
    ...

ex_id_list = df['externalid'].unique()

# 遍历所有external_id
for ex_id in ex_id_list:
    # 创建当前external_id的列表，用于储存rri
    rri_data = []
    # 筛选当前external_id的子表
    df_ex = df[df['externalid'] == ex_id]
    # 遍历子表中每一行的rri_data
    for index, df_line in df_ex.iterrows():
        rri_list = json.loads(df_line['rriData'].replace("'", '"'))
        for rri in rri_list:
            rri_data.append({
                'rri': rri['rri']['value'], 
                'sqi': rri['sqi'], 
                # 'time': timestamp2time(rri['timeFrame']['timestamp'])})
                'timestamp': rri['timeFrame']['timestamp']})
    # 将初步筛选出有效信息的数据以external_id存储到文件
    df_rri = pd.DataFrame(rri_data)
    print(f'ex_id: {ex_id}, count: {len(rri_data)}')
    df_rri.to_csv(f"{OUTPUT_FILE_PATH}/{file_name}/{ex_id}.csv", encoding="utf-8", index=False)

ex_id: 54, count: 1410005
ex_id: 45, count: 1351030
ex_id: 10, count: 1575571
ex_id: 31, count: 1521764
ex_id: 40, count: 1554431
ex_id: 29, count: 1158815
ex_id: 22, count: 116032
ex_id: 23, count: 1938705
ex_id: 43, count: 544593
ex_id: 21, count: 1432674
ex_id: 46, count: 368717
ex_id: 32, count: 1920643
ex_id: 18, count: 2569
ex_id: 58, count: 1519413
ex_id: 57, count: 1967219
ex_id: 42, count: 1217586
ex_id: 55, count: 1739964
ex_id: 48, count: 1521330
ex_id: 44, count: 1764957
ex_id: 3, count: 1350301
ex_id: 4, count: 1212519
ex_id: 52, count: 1594045
ex_id: 59, count: 1367138
ex_id: 14, count: 1278426
ex_id: 1, count: 1631321
ex_id: 27, count: 2450267
ex_id: 41, count: 829148
ex_id: 49, count: 984196
ex_id: 34, count: 1957930
ex_id: 15, count: 1681018
ex_id: 35, count: 267623
ex_id: 13, count: 1242617
ex_id: 56, count: 1049494
ex_id: 51, count: 342885
ex_id: 16, count: 946853
ex_id: 38, count: 962846
ex_id: 2, count: 181596
ex_id: 33, count: 774774
ex_id: 28, count: 1338228
ex_i

### 抽取量表、EMA的有效数据

In [ ]:
pd.set_option('display.max_rows', 100)

In [4]:
dtype = {"手环编号": int, "抑郁得分": int, "焦虑得分": int, "压力得分": int, "失眠得分": int, "自杀得分": int}
[df_before, df_after] = [pd.read_excel(_, dtype=str).drop('创建时间', axis=1).astype(dtype).sort_values(by='手环编号') for _ in LABEL_FILE_PATH_LIST]
df_before = df_before.rename(columns={'手环编号': 'external_id', '抑郁得分': 'PHQ9-1', '焦虑得分': 'GAD7-1', '压力得分': 'ISI-1', '失眠得分': 'PSS-1', '自杀得分': 'BSI-1'})
df_after = df_after.rename(columns={'手环编号': 'external_id', '抑郁得分': 'PHQ9-2', '焦虑得分': 'GAD7-2', '压力得分': 'ISI-2', '失眠得分': 'PSS-2', '自杀得分': 'BSI-2'})
df = pd.merge(df_before, df_after)
df.to_csv(f"{LABEL_FILE_PATH}/scale.csv", encoding="utf-8", index=False)

In [5]:
dtype = {"external_id": int, "upload_time": str}
df_ema = pd.read_csv(LABEL_FILE_PATH_EMA, encoding="utf-8", dtype=str).drop(["no", "sleep_time", "awake_time"], axis=1).astype(dtype).sort_values(by='external_id')
df_ema['upload_time'] = pd.to_datetime(df_ema['upload_time'])
df_ema = df_ema[(df_ema['upload_time'] >= BEGIN_TIME) & (df_ema['upload_time'] <= END_TIME)]
df_ema = df_ema.sort_values(by=['external_id', 'upload_time'])
df_ema.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df_ema['date'] = df_ema['upload_time'].dt.date
df_ema['period'] = df_ema['upload_time'].dt.hour.apply(lambda x: 1 if x > 12 else 0)
df_ema = df_ema.drop('upload_time', axis=1)
df_ema.to_csv(f"{LABEL_FILE_PATH}/ema.csv", encoding="utf-8", index=False)

 ### 统计数据

In [3]:
file_name_list = os.listdir(OUTPUT_FILE_PATH)
for file_name in file_name_list:
    ex_id_list = [int(_.split('.')[0]) for _ in os.listdir(f"{DATA_FILE_PATH}/{file_name}")]
    ex_id_list.sort()
    for ex_id in tqdm(ex_id_list):
        df_data = pd.read_csv(f"{DATA_FILE_PATH}/{file_name}/{ex_id}.csv", encoding="utf-8").drop('timestamp', axis=1)

['continuousbloodoxygensaturation',
 'continuousheartrate',
 'continuousrri',
 'dailyworkoutdetail']